In [4]:
import os
import json
from typing import List
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import AzureCosmosDBVectorSearch
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.messages import SystemMessage

from dotenv import load_dotenv

load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-3-small"
COMPLETIONS_DEPLOYMENT_NAME = "gpt-4"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2024-02-01"

In [5]:
# Establish Azure OpenAI connectivity
llm = AzureChatOpenAI(            
        temperature = 0,
        openai_api_version = AOAI_API_VERSION,
        azure_endpoint = AOAI_ENDPOINT,
        openai_api_key = AOAI_KEY,         
        azure_deployment = COMPLETIONS_DEPLOYMENT_NAME
)
embedding_model = AzureOpenAIEmbeddings(
    openai_api_version = AOAI_API_VERSION,
    azure_endpoint = AOAI_ENDPOINT,
    openai_api_key = AOAI_KEY,   
    azure_deployment = EMBEDDINGS_DEPLOYMENT_NAME,
    chunk_size=10
)

/Users/andrewbovey/.pyenv/versions/3.11.8/envs/cmsopengpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [7]:
# Reference the existing vector store
vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string = CONNECTION_STRING,
    namespace = "cms_open.cms_open",
    embedding = embedding_model,
    index_name = "VectorSearchIndex",    
    embedding_key = "contentVector",
    text_key = "_id"
)

In [8]:
# A system prompt describes the responsibilities, instructions, and persona of the AI.
# Note the addition of the templated variable/placeholder for the list of products and the incoming question.
system_prompt = """Extract and summarize relevant parts of the documents.
                Review the document. There are many different topics covered in the document. Decide if each topic is relevant to the user. If it is relevant, write it out a title and a brief summary of less than 5 words. Here is an example: Diabetes Management: strategies for managing diabetes
                Then ask the user which of the topics they are interested in"
Documents:
{documents}

Question:
{question}
"""

In [9]:
# remember that each Document contains a page_content property
# that is populated with the _id field of the document
# all other document fields are located in the metadata property
def format_docs(docs:List[Document]) -> str:
        """
        Prepares the product list for the system prompt.
        """
        str_docs = []
        for doc in docs:
                # Build the product document without the contentVector
                doc_dict = {"_id": doc.page_content}
                doc_dict.update(doc.metadata)
                if "contentVector" in doc_dict:  
                        del doc_dict["contentVector"]
                str_docs.append(json.dumps(doc_dict, default=str))                  
        # Return a single string containing each product JSON representation
        # separated by two newlines
        return "\n\n".join(str_docs)

In [10]:
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create the prompt template from the system_prompt text
llm_prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    # populate the tokens/placeholders in the llm_prompt 
    # products takes the results of the vector store and formats the documents
    # question is a passthrough that takes the incoming question
    { "documents": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    # pass the populated prompt to the language model
    | llm
    # return the string ouptut from the language model
    | StrOutputParser()
)

In [11]:
question = "Give me a quick summary of the documents"
response = rag_chain.invoke(question)
print(response)

1. Federal Register Proposed Rules: Document contains proposed rules and regulations. 
2. Medicare Program; Hospital Inpatient Prospective: Discusses changes to the Medicare program and hospital inpatient prospective.
3. Quicktome Software Suite: Discusses a new software suite for mapping the human brain.
4. Changes for Hospitals Excluded From the IPPS: Discusses changes for hospitals excluded from the Inpatient Prospective Payment System.
5. Quality Data Reporting Requirements: Discusses requirements for quality data reporting for specific providers and suppliers.
6. Other Provisions Included in This Final Rule: Discusses other provisions included in the final rule, including rural emergency hospitals and physician self-referral.
7. MedPAC Recommendations and Publicly Available Files: Discusses recommendations from the Medicare Payment Advisory Commission and publicly available files. 

Please let me know which topic you are interested in.
